In [1]:
from PIL import Image, ImageChops, ImageEnhance
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from pylab import *
from sklearn.metrics import classification_report, precision_score
import pandas as pd
import os
from PIL.ExifTags import TAGS

Using TensorFlow backend.


In [2]:
def convert_to_ela_image(path, quality):
    filename = path
    resaved_filename = filename.split('.')[0] + '.resaved.jpg'
    ELA_filename = filename.split('.')[0] + '.ela.png'
    
    im = Image.open(filename).convert('RGB')
    im.save(resaved_filename, 'JPEG', quality=quality)
    resaved_im = Image.open(resaved_filename)
    
    ela_im = ImageChops.difference(im, resaved_im)
    
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    
    return ela_im

In [3]:
from skimage import transform
def load(filename):
    image = Image.open(filename)
    np_image = convert_to_ela_image(filename, 90)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (128, 128, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [4]:
# model = load_model('saved-models/model-v1-with-casia.h5')
# model = load_model('saved-models/model-v2-with-mixed-dataset.h5')
# model = load_model('saved-models/model-v3-with-real-fake-dataset.h5')
model = load_model('../server/saved-models/model-v4-with-adam-optimizer.h5')

In [5]:
def predict_image(label):
    if label:
        return "Tampered Image"
    return "Real Image"

In [227]:
def metadata_extraction(filepath):
    try:
        f=1
        img = Image.open(filepath)
        info = img._getexif()
        if info:
            for (tag, value) in info.items():
                if "Software" == TAGS.get(tag, tag):
                    print("Fake Image")
                    print("Found Software Signature : " + value)
                    f=0
                    return 1

        if f:
            print("No Software Signature Found")
            return 0
    except Exception as e:
        print("Failed to Load Metadata : "+str(e))

In [6]:
dataset = pd.read_csv('datasets/doku-wallet/doku-wallet.csv')

In [7]:
X = []
Y = []
for index, row in dataset.iterrows():
    if os.path.isfile(row[0]):
        X.append(row[0])
        Y.append(1)
        print('done: '+str(index))
print("==== FINISHED ====")

done: 0
done: 1
done: 2
done: 3
done: 4
done: 5
done: 6
done: 7
done: 8
done: 9
done: 10
done: 11
done: 12
done: 13
done: 14
done: 15
done: 16
done: 17
done: 18
done: 19
done: 20
done: 21
==== FINISHED ====


### BERIKUT ADALAH HASIL DARI MODEL V2 DENGAN MENGGUNAKAN TEST DARI DATA MASJAY
#### Accuracy: 16/22

In [8]:
index = 0
for u in range(len(X)):
    if Y[u]==1:
        print('++++ TAMPERED IMAGE ++++')
    else:
        print('++++ Real IMAGE ++++')
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

++++ TAMPERED IMAGE ++++
Hasil Predict:  1
Result for  1159846246.png :  Tampered Image ( 97.25062847137451 %),  Real Image ( 2.749364823102951 %)

++++ TAMPERED IMAGE ++++
Hasil Predict:  0
Result for  selfie 1678548433.png :  Real Image ( 97.69407510757446 %),  Tampered Image ( 2.3059330880641937 %)

++++ TAMPERED IMAGE ++++
Hasil Predict:  1
Result for  1014396712.png :  Tampered Image ( 97.70752191543579 %),  Real Image ( 2.2924788296222687 %)

++++ TAMPERED IMAGE ++++
Hasil Predict:  0
Result for  1023702304aa.png :  Real Image ( 66.57311916351318 %),  Tampered Image ( 33.42687785625458 %)

++++ TAMPERED IMAGE ++++
Hasil Predict:  1
Result for  1017728442a.jpg :  Tampered Image ( 78.18999886512756 %),  Real Image ( 21.80999666452408 %)

++++ TAMPERED IMAGE ++++
Hasil Predict:  0
Result for  selfie 1705208902.png :  Real Image ( 97.77804017066956 %),  Tampered Image ( 2.2219600155949593 %)

++++ TAMPERED IMAGE ++++
Hasil Predict:  0
Result for  1023599637.jpg :  Real Image ( 58.389

### BERIKUT ADALAH HASIL DARI MODEL V1 DENGAN MENGGUNAKAN TEST DARI CUSTOM DATA
#### Accuracy: 21/31

In [211]:
benar = 0
salah = 0
index = 0
for u in range(len(X)):
    if Y[u]==1:
        tampered+=1
        print('++++ TAMPERED IMAGE ++++')
    else:
        real+=1
        print('++++ Real IMAGE ++++')
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        if i[1] == Y[index]:
            benar+=1
        else:
            salah+=1
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-11-custom.jpeg :  Real Image ( 93.50712895393372 %),  Tampered Image ( 6.492865830659866 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-12-custom.jpeg :  Real Image ( 60.21862030029297 %),  Tampered Image ( 39.78138267993927 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-13-custom.jpeg :  Real Image ( 94.27407383918762 %),  Tampered Image ( 5.725929141044617 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-14-custom.jpeg :  Real Image ( 78.77391576766968 %),  Tampered Image ( 21.22608721256256 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-15-custom.jpg :  Real Image ( 65.74325561523438 %),  Tampered Image ( 34.256747364997864 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-16-custom.jpg :  Tampered Image ( 73.39617013931274 %),  Real Image ( 26.603829860687256 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-17-custom.jpg :  Tampered Image ( 73.0123162269592

In [212]:
print("BENAR: ",benar)
print("SALAH: ",salah)

BENAR:  20
SALAH:  11


### <span style="color:red; font-weight: bold">BERIKUT ADALAH HASIL DARI MODEL V2 DENGAN MENGGUNAKAN TEST DARI CUSTOM EDITED DATA DENGAN IMAGE QUALITY 10 </span>
#### Accuracy: 21/31
#### <b>Real Image Well Predicted: 14/16</b>
#### Fake Image Well Predicted: 7/15

In [214]:
benar = 0
salah = 0
index = 0
for u in range(len(X)):
    if Y[u]==1:
        print('++++ TAMPERED IMAGE ++++')
        tampered+=1
    else:
        real+=1
        print('++++ Real IMAGE ++++')
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        if i[1] == Y[index]:
            benar+=1
        else:
            salah+=1
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-11-custom.jpeg :  Real Image ( 64.3819808959961 %),  Tampered Image ( 35.61801314353943 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-12-custom.jpeg :  Real Image ( 78.4589171409607 %),  Tampered Image ( 21.541081368923187 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-13-custom.jpeg :  Real Image ( 95.52698731422424 %),  Tampered Image ( 4.473015293478966 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-14-custom.jpeg :  Real Image ( 77.88512706756592 %),  Tampered Image ( 22.11487889289856 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-15-custom.jpg :  Real Image ( 94.55111622810364 %),  Tampered Image ( 5.448887124657631 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-16-custom.jpg :  Real Image ( 77.69573926925659 %),  Tampered Image ( 22.30425477027893 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-17-custom.jpg :  Tampered Image ( 92.13294386863708 %

In [215]:
print("BENAR: ",benar)
print("SALAH: ",salah)

BENAR:  21
SALAH:  10


### <span style="color:red; font-weight: bold">BERIKUT ADALAH HASIL DARI MODEL V2 DENGAN MENGGUNAKAN TEST DARI CUSTOM EDITED DATA DENGAN IMAGE QUALITY 90 </span>
#### Accuracy: 22/31
#### Real Image Well Predicted: 8/16
#### <b>Fake Image Well Predicted: 14/15</b>

In [217]:
benar = 0
salah = 0
index = 0
for u in range(len(X)):
    if Y[u]==1:
        print('++++ TAMPERED IMAGE ++++')
        tampered+=1
    else:
        real+=1
        print('++++ Real IMAGE ++++')
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        if i[1] == Y[index]:
            benar+=1
        else:
            salah+=1
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-11-custom.jpeg :  Real Image ( 99.99996423721313 %),  Tampered Image ( 3.858316972582543e-05 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-12-custom.jpeg :  Tampered Image ( 61.81615591049194 %),  Real Image ( 38.18384110927582 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-13-custom.jpeg :  Real Image ( 91.62117838859558 %),  Tampered Image ( 8.378823846578598 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-14-custom.jpeg :  Real Image ( 99.97339844703674 %),  Tampered Image ( 0.026605598395690322 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-15-custom.jpg :  Tampered Image ( 61.06429100036621 %),  Real Image ( 38.93571197986603 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-16-custom.jpg :  Real Image ( 89.0152096748352 %),  Tampered Image ( 10.984792560338974 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-17-custom.jpg :  Tampered Image ( 99.89207983

In [218]:
print("BENAR: ",benar)
print("SALAH: ",salah)

BENAR:  22
SALAH:  9


### BERIKUT ADALAH HASIL DARI MODEL V3 DENGAN MENGGUNAKAN TEST DARI CUSTOM EDITED DATA
#### Accuracy: 17/31

In [180]:
benar = 0
salah = 0
index = 0
for u in range(len(X)):
    if Y[u]==1:
        tampered+=1
        print('++++ TAMPERED IMAGE ++++')
    else:
        real+=1
        print('++++ Real IMAGE ++++')
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        if i[1] == Y[index]:
            benar+=1
        else:
            salah+=1
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-11-custom.jpeg :  Tampered Image ( 100.0 %),  Real Image ( 1.012282383703331e-12 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-12-custom.jpeg :  Tampered Image ( 99.80911612510681 %),  Real Image ( 0.19088619155809283 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-13-custom.jpeg :  Tampered Image ( 99.0831196308136 %),  Real Image ( 0.9168803691864014 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-14-custom.jpeg :  Tampered Image ( 99.99964237213135 %),  Real Image ( 0.00036005958463647403 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-15-custom.jpg :  Tampered Image ( 98.14520478248596 %),  Real Image ( 1.8547967076301575 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-16-custom.jpg :  Tampered Image ( 100.0 %),  Real Image ( 1.1602598866216045e-12 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-17-custom.jpg :  Tampered Image ( 99.9996542930603 %),  Rea

In [181]:
print("BENAR: ",benar)
print("SALAH: ",salah)

BENAR:  17
SALAH:  14


### BERIKUT ADALAH HASIL DARI MODEL V4 DENGAN MENGGUNAKAN TEST DARI CUSTOM EDITED DATA
#### Accuracy: 19/31

In [177]:
benar = 0
salah = 0
index = 0
for u in range(len(X)):
    if Y[u]==1:
        print('++++ TAMPERED IMAGE ++++')
    else:
        print('++++ Real IMAGE ++++')
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        if i[1] == Y[index]:
            benar+=1
        else:
            salah+=1
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-11-custom.jpeg :  Real Image ( 100.0 %),  Tampered Image ( 2.6969558856136666e-11 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-12-custom.jpeg :  Real Image ( 99.16306138038635 %),  Tampered Image ( 0.8369325660169125 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-13-custom.jpeg :  Real Image ( 87.34590411186218 %),  Tampered Image ( 12.654094398021698 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-14-custom.jpeg :  Real Image ( 99.99994039535522 %),  Tampered Image ( 6.080412049414008e-05 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-15-custom.jpg :  Real Image ( 83.53263735771179 %),  Tampered Image ( 16.467367112636566 %)

++++ Real IMAGE ++++
Hasil Predict:  0
Result for  real-16-custom.jpg :  Real Image ( 73.2312560081482 %),  Tampered Image ( 26.768746972084045 %)

++++ Real IMAGE ++++
Hasil Predict:  1
Result for  real-17-custom.jpg :  Tampered Image ( 93.39426755905151 

In [178]:
print("BENAR: ",benar)
print("SALAH: ",salah)

BENAR:  19
SALAH:  12


In [230]:
benar = 0
salah = 0
index = 0
for u in range(len(X)):
    if Y[u]==1:
        print('++++ TAMPERED IMAGE ++++')
        tampered+=1
    else:
        real+=1
        print('++++ Real IMAGE ++++')
    print("Level 1: Metadata Extraction:\n")
    metadata = metadata_extraction(X[u])
    print("Level 2: ELA Analysis:\n")
    img = load(X[u])
    for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
        print("Hasil Predict: ",i[1])
        if i[1] == Y[index]:
            benar+=1
        else:
            salah+=1
        index+=1
        print("Result for ",X[u].split('/')[2],": ", predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")
    
    print("Summary of 2 Levels Analysis: ", metadata and model.predict_classes(img, batch_size=50), end="\n\n\n")

++++ Real IMAGE ++++
Level 1: Metadata Extraction:

No Software Signature Found
Level 2: ELA Analysis:

Hasil Predict:  0
Result for  real-11-custom.jpeg :  Real Image ( 99.99996423721313 %),  Tampered Image ( 3.858316972582543e-05 %)

Summary of 2 Levels Analysis:  0


++++ Real IMAGE ++++
Level 1: Metadata Extraction:

No Software Signature Found
Level 2: ELA Analysis:

Hasil Predict:  1
Result for  real-12-custom.jpeg :  Tampered Image ( 61.81615591049194 %),  Real Image ( 38.18384110927582 %)

Summary of 2 Levels Analysis:  0


++++ Real IMAGE ++++
Level 1: Metadata Extraction:

No Software Signature Found
Level 2: ELA Analysis:

Hasil Predict:  0
Result for  real-13-custom.jpeg :  Real Image ( 91.62117838859558 %),  Tampered Image ( 8.378823846578598 %)

Summary of 2 Levels Analysis:  0


++++ Real IMAGE ++++
Level 1: Metadata Extraction:

No Software Signature Found
Level 2: ELA Analysis:

Hasil Predict:  0
Result for  real-14-custom.jpeg :  Real Image ( 99.97339844703674 %),  Ta

In [8]:
img = load('../datasets/custom-edited/real-2-custom.jpg')
for i in zip(model.predict_proba(img, batch_size=50), model.predict_classes(img, batch_size=50), model.predict(img, batch_size=50)):
    print("Hasil Predict: ",i[1])
    print(predict_image(i[1]),"(",max(i[0])*100,"%), ", predict_image(not i[1]),"(",min(i[0])*100,"%)", end="\n\n")

Hasil Predict:  1
Tampered Image ( 99.57030415534973 %),  Real Image ( 0.42969677597284317 %)

